In [1]:
from __future__ import annotations
from pathlib import Path
from datetime import datetime, timedelta, timezone
import json, os, subprocess, requests, jwt, time
import pandas as pd

In [2]:
BASE_URL = "https://m-path.io/API2" # Set up base URL
USER_CODE = os.getenv("MPATH_USERCODE", "ukmp2") # Set up user code.
PRIVATE_KEY_PEM = Path.home() / ".mpath_private_key.pem"
PUBLIC_KEY_PEM = Path.home() / ".mpath_public_key.pem"
BASE_DUMP_DIR = Path("mpath_raw").expanduser()
CONNECTION_ID = "290982" # Set up connection ID

In [3]:
import os
from pathlib import Path

# Set up base URL
if not BASE_URL.startswith("https://"):
    print("❌ BASE_URL is not valid.")

# Load user code from environment
if not USER_CODE:
    print("❌ USER_CODE is not valid.")

# Check private/public key file paths
if not PRIVATE_KEY_PEM.exists():
    print(f"❌ Private key not found: {PRIVATE_KEY_PEM}")

if not PUBLIC_KEY_PEM.exists():
    print(f"❌ Public key not found: {PUBLIC_KEY_PEM}")

# Check base dump directory
try:
    BASE_DUMP_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"❌ Failed to create BASE_DUMP_DIR: {BASE_DUMP_DIR} - {e}")

# Check connection-specific directory
try:
    CONN_DIR = BASE_DUMP_DIR / str(CONNECTION_ID)
    CONN_DIR.mkdir(parents=True, exist_ok=True)
except Exception as e:
    print(f"❌ Failed to create CONN_DIR: {CONN_DIR} - {e}")

In [4]:
from get_data import get_data,flatten_and_save
# Download a raw datset
rows, conn_dir = get_data(user_code=USER_CODE,
                          connection_id=CONNECTION_ID,
                          private_key_path=Path(PRIVATE_KEY_PEM),
                          base_dump_dir=Path(BASE_DUMP_DIR)
                          )

# clean and flattened csv file generation
df, _ = flatten_and_save(raw_rows=rows, connection_id=CONNECTION_ID, conn_dir=conn_dir,tz= "US/Eastern")
display(df.head())

API returned status –1 (attempt 1/3); retrying in 5 seconds.
✓ Raw payload saved → mpath_raw/290982/data_290982_20250801T173319Z.json
✓ Clean CSV saved → mpath_raw/290982/data_clean_290982_20250801T173319Z_11rows.csv


,id,timeStampStop,timeStampStart,sentBeepId,downloadedAt,data_timeStampStart,data_timeStampStop,data_beepId,data_carerId,data_sentToDatabase,...,label_BCoq_basicQuestion_imageLocations,label_BCoq_basicQuestion_imageWidth,label_BCoq_basicQuestion_keepMedia,label_BCoq_basicQuestion_showLast,label_BCoq_basicQuestion_options,label_BCoq_basicQuestion_showMidpoint,label_BCoq_basicQuestion_anchorMiddle,label_BCoq_basicQuestion_verticalSlider,label_BCoq_basicQuestion_startSlider,label_BCoq_value
0,10159561,2025-06-06 08:32:32,2025-06-06 08:32:15,24358525,20250801T173319Z,2025-06-06 12:32:15,2025-06-06 12:32:32,24358525,324906,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10247321,2025-06-24 07:14:08,2025-06-24 07:13:48,24580673,20250801T173319Z,2025-06-24 11:13:48,2025-06-24 11:14:08,24580673,324906,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10247466,2025-06-24 07:51:23,2025-06-24 07:51:13,24580936,20250801T173319Z,2025-06-24 11:51:13,2025-06-24 11:51:23,24580936,324906,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10252624,2025-06-25 11:02:43,2025-06-25 11:02:24,24594522,20250801T173319Z,2025-06-25 15:02:24,2025-06-25 15:02:43,24594522,324906,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10259176,2025-06-27 05:04:03,2025-06-27 05:00:13,24611866,20250801T173319Z,2025-06-27 09:00:13,2025-06-27 09:04:03,24611866,324906,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from get_interactions import get_interactions
dfs = get_interactions(
        connection_id=CONNECTION_ID,         # participant / connection ID
        user_code=USER_CODE,             # your 5-character practitioner code
        private_key_path=Path(PRIVATE_KEY_PEM),
        out_base=Path("interactions_raw").expanduser()
)
# Inspect the returned DataFrames
print(dfs.keys())

Fetching interactions for connection 290982 …
status –1; retrying … [1/3]
✓ Raw JSON saved → interactions_raw/290982/interactions_290982_20250801T173325Z.json
  └─ root 1: 3 questions → interactions_raw/290982/01_Demo_survey_20250801T173325Z_3rows.csv
  └─ root 2: 3 questions → interactions_raw/290982/02_New_interaction_20250801T173325Z_3rows.csv
  └─ root 3: 3 questions → interactions_raw/290982/03_New_interaction2_20250801T173325Z_3rows.csv
  └─ root 4: 3 questions → interactions_raw/290982/04_New_interaction_lauren_20250801T173325Z_3rows.csv
  └─ root 5: 1 questions → interactions_raw/290982/05_New_interaction_20250801T173325Z_1rows.csv
dict_keys(['Demo survey', 'New interaction', 'New interaction2', 'New interaction_lauren'])


In [6]:
from get_schedule import get_schedule   # path / module name as you saved it
df_sched = get_schedule(
    connection_id=CONNECTION_ID,   # the participant / connection ID (int)
    user_code=USER_CODE,            # your 5-character practitioner code
    private_key_path=Path(PRIVATE_KEY_PEM),
    out_base=Path("interactions_raw").expanduser()
)

# Inspect the returned DataFrame
print(df_sched.shape)   # rows, columns
df_sched.head()         # first few rows


Fetching schedule for connection 290982 …
status –1; retrying … [1/3]
✓ Raw JSON saved → interactions_raw/290982/schedule_290982_20250801T173331Z.json
✓ CSV saved → interactions_raw/290982/schedule_290982_20250801T173331Z_39rows.csv
(39, 19)


,startTime,endTime,scheduledTime,itemId,sentTime,resultReceivedTime,beepId,randomizationScheme,reminderIntervals,expirationInterval,useAsButton,buttonText,buttonLabel,singleUse,required,passed,reminderForScheduledBeepId,scheduleType,seed
0,2025-06-24 11:50:00,2025-06-24 11:53:00,2025-06-24 11:51:06,u75qAtkZlr8DJ3V3,2025-06-24 17:51:07,2025-06-24 17:51:24,35795914,1,[1800],3600.0,0,None,None,1,0,1,0,0.0,None
1,2025-06-25 15:00:00,2025-06-25 17:00:00,2025-06-25 15:00:00,N1m7ygNkbTTi6N8D,2025-06-25 21:00:03,2025-06-25 21:02:43,35810026,0,[3600],3600.0,0,None,None,1,0,1,0,0.0,None
2,2025-06-26 15:00:00,2025-06-26 17:00:00,2025-06-26 15:00:00,N1m7ygNkbTTi6N8D,2025-06-26 21:00:06,None,35810027,0,[3600],3600.0,0,None,None,1,0,1,0,0.0,None
3,2025-06-27 15:00:00,2025-06-27 17:00:00,2025-06-27 15:00:00,N1m7ygNkbTTi6N8D,2025-06-27 21:00:08,None,35810028,0,[3600],3600.0,0,None,None,1,0,1,0,0.0,None
4,2025-06-28 15:00:00,2025-06-28 17:00:00,2025-06-28 15:00:00,N1m7ygNkbTTi6N8D,2025-06-28 21:00:07,None,35810029,0,[3600],3600.0,0,None,None,1,0,1,0,0.0,None


In [7]:
# from get_schedule import get_schedule   # path / module name as you saved it
# df = get_schedule(
#     connection_id=CONNECTION_ID,   # the participant / connection ID (int)
#     user_code=USER_CODE            # your 5-character practitioner code
# )
# display(df)

In [8]:
import pandas as pd, pytz          # pip install pytz if you don’t have it

# df  ←– DataFrame returned by get_schedule()
# ──────────────────────────────────────────────────────────────────────
# 1) Identify any timestamp-like columns that came back as strings
df = df_sched
ts_cols = [
    c for c in df.columns
    if c in ("scheduledTime", "startTime", "timeStart", "timeEnd")  # add more if needed
       and df[c].notna().any()
]

# 2) Parse those strings → timezone-aware pandas Timestamps
eastern = pytz.timezone("US/Eastern")            # same zone get_schedule() used
for c in ts_cols:
    df[c] = pd.to_datetime(df[c]).dt.tz_localize(eastern)

# 3) Build a mask for “any timestamp in the future”
now = pd.Timestamp.now(tz=eastern)
future_mask = pd.Series(False, index=df.index)
for c in ts_cols:
    future_mask |= df[c] > now

# 4) Slice the frame
df_future = df[future_mask].copy().reset_index(drop=True)

print(f"Existing future beeps: {len(df_future)}")
display(df_future)



Existing future beeps: 0


,startTime,endTime,scheduledTime,itemId,sentTime,resultReceivedTime,beepId,randomizationScheme,reminderIntervals,expirationInterval,useAsButton,buttonText,buttonLabel,singleUse,required,passed,reminderForScheduledBeepId,scheduleType,seed


In [9]:
from merge_and_push_schedule import build_entries   # the helper you showed earlier

ITEM_ID = "N1m7ygNkbTTi6N8D"   # example m-Path item (survey) ID
CONN    = CONNECTION_ID

new_beeps = build_entries(
    starts  = ["2025-07-28 09:00:00", "2025-07-29 21:00:00"],    # local time strings
    ends    = ["2025-07-28 10:00:00", "2025-07-29 22:00:00"],
    item_id = ITEM_ID,
    labels  = ["jul24_morning2", "jul24_evening2"],              # becomes localId
    reminder_intervals = [1800],                                 # optional
)
print(f"Built {len(new_beeps)} new entries")
new_beeps[0]                                   # peek at the first dict


Built 2 new entries


{'startTime': '2025-07-28 09:00:00',
 'scheduledTime': '2025-07-28 09:00:00',
 'itemId': 'N1m7ygNkbTTi6N8D',
 'beepId': 0,
 'localId': 'jul24_morning2',
 'randomizationScheme': 0,
 'endTime': '2025-07-28 10:00:00',
 'reminderIntervals': [1800]}

In [10]:
from schedule_json_builder import combine_entries, save_upload_json
import pandas as pd

# df_future and new_beeps already defined in your earlier cells
combined = combine_entries(df_future=df_future, new_beeps=new_beeps)

print(f"Final list: {len(combined)} beeps "
      f"({sum('localId' in e for e in combined)} with localId)")

json_path = save_upload_json(combined)
print("✓ Upload-ready JSON →", json_path)

# Optional: inspect as a DataFrame for readability
display(pd.DataFrame(combined))

Final list: 2 beeps (2 with localId)
✓ Upload-ready JSON → /Users/leek13/github/EMA-API-Handling/upload_ready_20250801T133331.json


,itemId,localId,endTime,startTime,beepId,scheduledTime,reminderIntervals,randomizationScheme
0,N1m7ygNkbTTi6N8D,jul24_morning2,2025-07-28 10:00:00,2025-07-28 09:00:00,0,2025-07-28 09:00:00,[1800],0
1,N1m7ygNkbTTi6N8D,jul24_evening2,2025-07-29 22:00:00,2025-07-29 21:00:00,0,2025-07-29 21:00:00,[1800],0


In [11]:
# --- Upload the schedule JSON you just created ---
from pathlib import Path
import json
from set_schedule_from_json import set_schedule  # import the function you already wrote

# Read the file we just saved
entries = json.loads(Path(json_path).read_text("utf-8"))

# Run the upload
reply = set_schedule(
    entries       = entries,
    user_code     = USER_CODE,
    connection_id = int(CONNECTION_ID),
    private_key_path=  PRIVATE_KEY_PEM,
    minimal       = True,   # or False, depending on what you want
    retries       = 3
)

print("Server reply:")
print(json.dumps(reply, indent=2, ensure_ascii=False))

# If the API returns the new2id mapping, show it nicely
if "new2id" in reply:
    print("\nMapping localId to new beepId:")
    print(json.dumps(reply["new2id"], indent=2, ensure_ascii=False))


Server reply:
{
  "status": -2
}


In [ ]:
# Optional (get client)
from pathlib import Path
from get_clients import MPathConfig, get_clients

cfg = MPathConfig(
    private_key_pem=Path.home() / Path(".mpath_private_key.pem"),
    public_key_pem=Path.home() / Path(".mpath_public_key.pem"),
    base_url="https://dashboard.m-path.io/API2",
    base_dump_dir=Path("~/mpath_downloads/clients").expanduser(),
)

rows, out_dir = get_clients(
    user_code="ukmp2",
    changed_after_utc="2019-07-01 00:00:00",
    show_url=True,
    config=cfg,
)

print(f"{len(rows)} rows downloaded to {out_dir}")
display(rows)  # peek

→ GET https://dashboard.m-path.io/API2/getClients?userCode=ukmp2&JWT=<redacted>&changedAfterUTC=2019-07-01+00%3A00%3A00
✓ Raw payload saved → /Users/leek13/mpath_downloads/clients/clients_2019-07-01_000000_20250801T173418Z.json
3 rows downloaded to /Users/leek13/mpath_downloads/clients


[{'connectionId': 296533,
  'alias': 'system test # JMM',
  'blocked': 0,
  'archived': 0,
  'compliance': None,
  'requestedUTC': '2025-08-01 16:33:52',
  'changedUTC': '2025-08-01 16:33:52',
  'unregistered': 0,
  'downloadedAt': '20250801T173418Z'},
 {'connectionId': 296447,
  'alias': 'system test # LH',
  'blocked': 0,
  'archived': 0,
  'compliance': None,
  'requestedUTC': '2025-07-31 17:40:05',
  'changedUTC': '2025-07-31 17:40:05',
  'unregistered': 0,
  'downloadedAt': '20250801T173418Z'}]